Артем Жук, 399 группа
### EM for PCA

В данном репозитории реализованы две вариации ЕМ-алогоритма для анализа главных компонент. Статья с описанием выкладок: http://www.machinelearning.ru/wiki/images/7/73/BMMO11_11.pdf
Интерфейсы классов следуют интерфейсу sklearn.decomposition.PCA.

#### EMPCA
Класс EMPCA является реализацией ЕМ алгоритма для нахожения подпространста пространства признаков, порожденного первыми несколькими главными компонентами. Для всей выборки все признаки должны быть известны.  
Эта реализация опирается на достаточно простые выкладки, а кроме того быстрее работает, чем более общая реализация EMPCAM.

In [1]:
from empca import EMPCA
from empca_missing import EMPCAM
from sklearn.decomposition import PCA
import numpy as np
from utils import random_model, gram_schmidt, span_in

Сгенерируем выборку. Метод random_model(N, D, d) генрирует выборку с распределением $W\mathcal{N}(0, I_d) + \mathcal{N}(0, \varepsilon I_D), W\in M(\mathbb{R})^{D\times d}$.  
gram_schmidt(X) применяет алгоритм Грама-Шмидта к строкам Х.

In [2]:
X, W, T = random_model(30, 6, 1)
print("W=\n", W)
print("X..=\n", X[:5])

W=
 [[ 0.80766511 -3.04284396 -7.51495146 -2.84949247  8.19801441  3.85179355]]
X..=
 [[  1.1084357   -1.30820071  -4.90895327  -1.29960592   5.21991388
    2.03144389]
 [  0.64571353  -0.07770269   0.33014706   0.96793423   0.51394842
   -1.09151779]
 [ -0.49807375   6.07875637  14.54139686   4.59016023 -15.15980614
   -7.01616932]
 [  0.6132104   -2.08348901  -4.39644375  -1.28185953   4.0174445
    2.21811061]
 [ -0.48082356  -1.57492028  -1.43134362  -1.17329032   3.92875899
    1.38834662]]


Создадим экземпляр EMPCA и запустим на нашей выборке. Поле components_ содержит ортнормированный базис из n_components главных векторов.

In [3]:
empca = EMPCA(n_components=1, n_iter=100)
empca.fit(X)
print('basis:\n', empca.components_)
print('original basis:\n', gram_schmidt(W))

basis:
 [[ 0.06202824 -0.25136142 -0.59813343 -0.21025565  0.65882458  0.31136646]]
original basis:
 [[ 0.06455146 -0.24319488 -0.60062157 -0.22774154  0.65521438  0.307849  ]]


Как видим, алгоритм довольно точно определил направляющий вектор прямой, на которой лежат точки. Попробуем для плоскости:

In [4]:
X, W, T = random_model(60, 7, 5)
empca = EMPCA(n_components=1, n_iter=100)
empca.fit(X)
print('basis:\n', empca.components_)
print('original basis:\n', gram_schmidt(W))

basis:
 [[-0.5489386  -0.49993188 -0.03774341  0.42446507  0.05264976  0.14070607
   0.49453935]]
original basis:
 [[ 0.23292482  0.39597175  0.37400723  0.41763673 -0.26639632  0.48520001
   0.41020057]
 [-0.3110128  -0.51832853  0.25675356  0.46399796  0.40561628 -0.14779748
   0.40868253]
 [ 0.36365973 -0.12515884 -0.06939373 -0.03267816  0.66467168  0.58498271
  -0.24941992]
 [ 0.56124689 -0.06540257 -0.1778197   0.62363422 -0.07967346 -0.39302888
  -0.31522278]
 [ 0.27302376  0.29935704  0.66674498 -0.27267756  0.34488873 -0.44427164
  -0.02481841]]


На этот раз сложнее проверить совпадение векторов. Однако для проверки корректности нам можно проверять не точное совпадение базисов, а совпадение подпростраств, на них натянутых. Для этого воспользуемся функцией span_in(A, B), проверяющей что строки А лежат в линейной оболочке строк B.

In [5]:
print('span_in: ', span_in(empca.components_, gram_schmidt(W)))

span_in:  True


Сравним работу нашего преобразователя со стандартным sklearn.decomposition.PCA. Для этого сравним explained_variance_ratio_. Это доля дисперсии, приходящаяся на компоненту.

In [1]:
X, W, T = random_model(100, 50, 10)
pca = PCA(n_components=5)
empca = EMPCA(n_components=5, n_iter=100)

pca.fit(X)
empca.fit(X)

print('pca ratio:\n', pca.explained_variance_ratio_)
print('empca ratio:\n', empca.explained_variance_ratio_)

NameError: name 'random_model' is not defined

Как видим, за 300 итерация численный алгоритм достаточно точно нашел 

#### EMPCAM
Этот класс реализует тот же алгоритм, но допускает пропуски в данных. 


Сгененрируем выборку с 20% пропущенных значений:

In [7]:
X, W, T = random_model(100, 10, 1, 0.2)
print("X=\n", X[:5])

X=
 [[ -9.47924135  -0.32400759  -7.913485   -11.91856188          nan
  -11.31533725  10.29606614  13.48421466  13.05817822 -14.13734944]
 [  1.00126044  -0.0626547           nan   2.09166539   0.36436707
           nan  -0.49722563  -1.63446161  -1.26188072   1.45114822]
 [  0.09880113  -0.06869992  -0.75409562  -0.97428673  -0.5677724
   -1.56886931  -0.70074102   0.25233626   1.02794643  -1.47462933]
 [ -7.02853431  -2.21500292  -4.68553497  -7.4146958   -3.22222511
   -6.93384712   7.77246359   8.1219654    8.58026932          nan]
 [ -1.19193611          nan  -0.59327046          nan          nan
   -2.91422686   1.40294254   2.24571149   1.98331364  -1.67194105]]


И применим к ней алгоритм: 

In [8]:
empcam = EMPCAM(n_components=1, n_iter=100)
empcam.fit(X)
A = empcam.components_
B = gram_schmidt(W)
print('found basis: ', A)
print('original basis: ', B)
d = A[0] - B[0]
print('A - B: ', d)
print('diff = ', np.linalg.norm(d))

found basis:  [[-0.31322081 -0.01709025 -0.2135308  -0.38589466 -0.12226821 -0.33903985
   0.31082277  0.42163399  0.38071988 -0.39731521]]
original basis:  [[ 0.2998815   0.02451015  0.21120959  0.35437946  0.12519969  0.36332475
  -0.32616408 -0.40678102 -0.38342535  0.41560488]]
A - B:  [-0.6131023  -0.04160039 -0.42474039 -0.74027412 -0.24746789 -0.7023646
  0.63698684  0.82841501  0.76414523 -0.81292009]
diff =  1.99934295796


Как видим, нам удалось довольно точно восстановить исходную зависимость. 

Теперь построим модель с большим числом латентных переменных и проверим, что найденное подпрастранство попадает в пространство, натянутое на образы латентных переменных.

In [9]:
X, W, T = random_model(100, 30, 5, 0.1)
empcam = EMPCAM(n_components=2, n_iter=200)
empcam.fit(X)
print('lies_inside: ', span_in(empcam.components_, gram_schmidt(W)))

lies_inside:  True
